# Module 2 - Clean Up AWS Resources

----

When you no longer need the Amazon Bedrock Knowledge Base that we have created in this workshop module, execute this notebook to clean up all the provisioned resources.

----

## Initialize relevant variables and clients

In [ ]:
%store -r s3_bucket_name aoss_encryption_policy aoss_network_policy aoss_access_policy aoss_collection bedrock_kb_id

In [ ]:
import boto3
import time

boto3_session = boto3.Session()
bedrock_agent_client = boto3_session.client('bedrock-agent', region_name=boto3_session.region_name)
aoss_client = boto3.client('opensearchserverless')
s3_client = boto3_session.client('s3', region_name=boto3_session.region_name)
iam_client = boto3.client("iam")

## Delete Bedrock Knowledge Base data sources

In [ ]:
# Fetch data source(s)associated with kb
response = bedrock_agent_client.list_data_sources(knowledgeBaseId=bedrock_kb_id)
data_sources_list = [ds_summary for ds_summary in response['dataSourceSummaries']]

for idx, ds in enumerate(data_sources_list):
    bedrock_agent_client.delete_data_source(dataSourceId=ds["dataSourceId"], knowledgeBaseId=ds["knowledgeBaseId"])
    time.sleep(10)

## Remove Bedrock Knowledge Bases and OpenSearch Collection

In [ ]:
# Fetch BKB execution role
response = bedrock_agent_client.get_knowledge_base(knowledgeBaseId=bedrock_kb_id)
bedrock_kb_role_name = response['knowledgeBase']['roleArn'].split("/")[-1]

In [ ]:
# Fetch all attched policies with kb execution role
kb_attached_role_policies_response = iam_client.list_attached_role_policies(
    RoleName=bedrock_kb_role_name)
kb_attached_role_policies = kb_attached_role_policies_response['AttachedPolicies']

In [ ]:
bedrock_agent_client.delete_knowledge_base(knowledgeBaseId=bedrock_kb_id)
aoss_client.delete_collection(id=aoss_collection['createCollectionDetail']['id'])
aoss_client.delete_access_policy(type="data", name=aoss_access_policy['accessPolicyDetail']['name'])
aoss_client.delete_security_policy(type="network", name=aoss_network_policy['securityPolicyDetail']['name'])
aoss_client.delete_security_policy(type="encryption", name=aoss_encryption_policy['securityPolicyDetail']['name'])

## Delete IAM Role and Policies

In [ ]:
for policy in kb_attached_role_policies:
    iam_client.detach_role_policy(
        RoleName=bedrock_kb_role_name,
        PolicyArn=policy['PolicyArn']
    )

In [ ]:
iam_client.delete_role(RoleName=bedrock_kb_role_name)

In [ ]:
for policy in kb_attached_role_policies:
    iam_client.delete_policy(PolicyArn=policy['PolicyArn'])

## Delete S3 Objects and Bucket

In [ ]:
objects = s3_client.list_objects(Bucket=s3_bucket_name)

if 'Contents' in objects:
    for obj in objects['Contents']:
        s3_client.delete_object(Bucket=s3_bucket_name, Key=obj['Key'])
        
s3_client.delete_bucket(Bucket=s3_bucket_name)